In [14]:
# !pip install PyPDF2
# !pip install pyhwp
!pip install pytesseract
!pip install requests beautifulsoup4

In [15]:
import os 
import faiss
import numpy as np
import requests
import json
from PyPDF2 import PdfReader
import subprocess
from tqdm import tqdm
import pytesseract
from PIL import Image
import zipfile
import xml.etree.ElementTree as ET
import re
from nltk.tokenize import sent_tokenize

In [16]:
# OLLAMA API 앤드포인트
OLLAMA_API = "http://localhost:11434/api"

In [17]:
def get_embedding(text):
    try:
        response = requests.post(f"{OLLAMA_API}/embeddings", 
                                 json={"model": "EEVE-Korean-10.8B", "prompt": text})
        response.raise_for_status()
        return np.array(response.json()['embedding'])
    except Exception as e:
        print(f"Error in get_embedding: {e}")
        return None

def generate_response(prompt):
    try:
        response = requests.post(f"{OLLAMA_API}/generate", 
                                 # temperature 낮은 값으로 설정하여 일관성 증가
                                 json={"model": "EEVE-Korean-10.8B", "prompt": prompt, "temperature": 0.2},  
                                 stream=True)
        response.raise_for_status()
        full_response = ""
        for line in response.iter_lines():
            if line:
                decoded_line = line.decode('utf-8')
                try:
                    json_response = json.loads(decoded_line)
                    if 'response' in json_response:
                        full_response += json_response['response']
                except json.JSONDecodeError:
                    print(f"Invalid JSON: {decoded_line}")
        return full_response
    except Exception as e:
        print(f"Error in generate_response: {e}")
        return None

In [18]:
# 문서 준비 및 임베딩 형성 : 파일 형식에 따라 다른 형식으로 데이터를 임베딩 해야함

# PDF 임베딩
def read_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

# HWP 임베딩
def read_hwp(file_path):
    try:
        result = subprocess.run(['hwp5txt', file_path], capture_output=True, text=True)
        return result.stdout
    except Exception as e:
        print(f"Error reading HWP file: {e}")
        return ""

# 이미지 파일 임베딩
def extract_text_from_image(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image, lang='kor')
    return text

# 한글 버젼 ppt. 기본적인 텍스트 추출만 가능. 복잡한 서식, 표, 이미지 등은 처리하지 않음.
def read_hwpx(file_path):
    try:
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            # 'Contents/section0.xml' 파일에서 텍스트 추출
            with zip_ref.open('Contents/section0.xml') as section_file:
                tree = ET.parse(section_file)
                root = tree.getroot()
                
                # 모든 텍스트 노드 추출
                text_nodes = root.findall(".//hp:t", namespaces={'hp': 'http://www.hancom.co.kr/hwpml/2011/paragraph'})
                
                # 텍스트 결합
                full_text = ' '.join(node.text for node in text_nodes if node.text)
                
                # 불필요한 공백 제거
                full_text = re.sub(r'\s+', ' ', full_text).strip()
                
                return full_text
    except Exception as e:
        print(f"Error reading HWPX file: {e}")
        return ""

In [19]:
def read_file(file_path):
    _, ext = os.path.splitext(file_path)
    try:
        if ext.lower() == '.pdf':
            return read_pdf(file_path)
        elif ext.lower() == '.hwp':
            return read_hwp(file_path)
        elif ext.lower() in ['.jpg', '.png', '.jpeg']:
            return extract_text_from_image(file_path)
        elif ext.lower() in ['.doc', '.docx']:
            return read_doc(file_path)
        elif ext.lower() == '.hwpx':
            return read_hwpx(file_path)
        else:
            with open(file_path, 'r', encoding='utf-8') as f:
                return f.read()
    except Exception as e:
        # 파일명과 오류 메시지를 출력합니다.
        print(f"Error processing file: {file_path} - {str(e)}")
        return None

documents = []
embeddings = []

file_list = os.listdir('/Users/dahyun/Desktop/hallym')
for filename in tqdm(file_list, desc="Processing files"):
    file_path = os.path.join('/Users/dahyun/Desktop/hallym', filename)
    text = read_file(file_path)
    if text:
        embedding = get_embedding(text)
        if embedding is not None:
            documents.append(text)
            embeddings.append(embedding)
if not documents or not embeddings:
    print("No documents or embeddings were processed successfully.")
    exit()
    
def read_doc(file_path):
    import docx2txt
    return docx2txt.process(file_path)

Processing files:  12%|███▍                       | 1/8 [00:03<00:25,  3.66s/it]

Error processing file: /Users/dahyun/Desktop/hallym/.DS_Store - 'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte


Processing files: 100%|███████████████████████████| 8/8 [00:16<00:00,  2.10s/it]


In [20]:
# 임베딩 결과 확인
documents

['\n<표>\n\n<표>\n\n',
 '체육시설 추첨 신청서\n\n\n<표>\n\n※ 신청자 정보는 추첨 관련 공지를 할 예정이오니, 정확하게 기재하여 주시기 바랍니다.\n',
 '성 명 전공: 학년: 학번: 자 기 소 개 서 1. 나의 가치관과 목표 2. 나의 전공분야를 선택한 이유와 미래 희망 3. 장학금을 신청한 이유',
 '《 전공·학과, 단과대학 ·스쿨교학팀 전화번호 , 이메일주소 》\n전화번호 (033-248-_______ ), 이메일주소 ( ________@hallym.ac.kr)\n구분 전화 이메일 구분 전화 이메일\n인문\n대학인문대학 교학팀\n(FAX033-248-1505)1500 de1500\n글로벌\n융합대학글로벌융합대학 교학팀\n(FAX033-248-2485)\n1880 de1880인문학부 - - 글로벌학부\n국어국문학전공 1510 de1510 글로벌비즈니스전공\n철학전공 1550 de1550 정보법과학전공\n사학전공 1570 de1570 한중통번역전공\n영어영문학과 1530 de1530 문화산업전공\n중국학과 1590 de1590 한국학전공\n일본학과 1610 de1610 융합과학수사학과 1980 de1980\n통합스쿨통합스쿨 교학팀\n(FAX033-248-3560)3551 de3550 러시아학과 1630 de1630\n사회\n과학\n대학사회과학대학 교학팀\n(FAX033-248-1702)1700 de1700미디어\n스쿨언론방송융합미디어전공3552~4 de1910디지털미디어콘텐츠전공\n심리학과 1720~1 de1720 반도체⋅\n디 스 플 레 이\n스 쿨반도체전공\n3555 de2050사회학과 1740 de1740 디스플레이전공\n사회복지학부1760,\n1769de1760미래융합\n스쿨디지털인문예술전공\n3556~7 de3553사회복지학전공 글로벌협력전공\n노인복지학전공 융합관광경영전공\n정치행정학과 1780 de1780의약신소재전공\n융합신소재공학전공\n광고홍보학과 1949 de1949\n의과대학의과대학 교학팀\n(FAX033-24

In [21]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

def extract_text_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 페이지에서 불필요한 부분을 제외하고 주요 텍스트를 추출
        for script in soup(["script", "style"]):
            script.extract()  # 자바스크립트와 CSS 제거
        
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk for chunk in chunks if chunk)
        
        return text
    except Exception as e:
        print(f"Error extracting text from URL: {url} - {e}")
        return None

# 예시 URL 리스트 (원하는 URL로 변경)
urls = [
    "https://www.hallym.ac.kr/hallym_univ/sub02/cP8/sCP2.html"
]

# 각 URL에서 텍스트를 가져와 documents 리스트에 추가
for url in tqdm(urls, desc="Processing URLs"):
    text = extract_text_from_url(url)
    if text:
        documents.append(text)

# 텍스트 출력 (임베딩 전에 텍스트 확인)
for i, doc in enumerate(documents):
    print(f"Document {i+1}:\n{doc}\n")

Processing URLs: 100%|████████████████████████████| 1/1 [00:00<00:00,  1.85it/s]

Document 1:

<표>

<표>



Document 2:
체육시설 추첨 신청서


<표>

※ 신청자 정보는 추첨 관련 공지를 할 예정이오니, 정확하게 기재하여 주시기 바랍니다.


Document 3:
성 명 전공: 학년: 학번: 자 기 소 개 서 1. 나의 가치관과 목표 2. 나의 전공분야를 선택한 이유와 미래 희망 3. 장학금을 신청한 이유

Document 4:
《 전공·학과, 단과대학 ·스쿨교학팀 전화번호 , 이메일주소 》
전화번호 (033-248-_______ ), 이메일주소 ( ________@hallym.ac.kr)
구분 전화 이메일 구분 전화 이메일
인문
대학인문대학 교학팀
(FAX033-248-1505)1500 de1500
글로벌
융합대학글로벌융합대학 교학팀
(FAX033-248-2485)
1880 de1880인문학부 - - 글로벌학부
국어국문학전공 1510 de1510 글로벌비즈니스전공
철학전공 1550 de1550 정보법과학전공
사학전공 1570 de1570 한중통번역전공
영어영문학과 1530 de1530 문화산업전공
중국학과 1590 de1590 한국학전공
일본학과 1610 de1610 융합과학수사학과 1980 de1980
통합스쿨통합스쿨 교학팀
(FAX033-248-3560)3551 de3550 러시아학과 1630 de1630
사회
과학
대학사회과학대학 교학팀
(FAX033-248-1702)1700 de1700미디어
스쿨언론방송융합미디어전공3552~4 de1910디지털미디어콘텐츠전공
심리학과 1720~1 de1720 반도체⋅
디 스 플 레 이
스 쿨반도체전공
3555 de2050사회학과 1740 de1740 디스플레이전공
사회복지학부1760,
1769de1760미래융합
스쿨디지털인문예술전공
3556~7 de3553사회복지학전공 글로벌협력전공
노인복지학전공 융합관광경영전공
정치행정학과 1780 de1780의약신소재전공
융합신소재공학전공
광고홍보학과 1949 de1949
의과대학의과대학 교학팀
(FAX033-242-2524) 25

In [22]:
# Faiss 인덱스 생성
dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

In [23]:
def rag_query(query, k=3):
    query_embedding = get_embedding(query)
    if query_embedding is None:
        return "Failed to generate query embedding."
    
    _, indices = index.search(np.array([query_embedding]), k)
    
    context = "\n".join([documents[i] for i in indices[0]])
    
    # 프롬프트에 한국말로 결과값 나오도록 설정
    # 소장님이 제안해주신대로 주어진 정보 내에서만 답변 해줄 수 있을지 고려
    prompt = f"""
    Context: {context}

    질문: {query}

    지시사항:
    1. 주어진 컨텍스트 정보만을 사용하여 답변하세요.
    2. 정확한 정보가 없다면 "이 정보는 주어진 컨텍스트에 없습니다"라고 말하세요.
    3. 답변은 중요 내용 위주로 간결하게 작성하세요.

    답변:
    """
    
#     # Few-shot 예제와 함께 프롬프트 생성
#     # 실행해봤으나 이 예제에 결과가 영향을 너무 많이 받음을 확인
#     few_shot_example = """
#     질문: 도서관 운영 시간은 어떻게 되나요?
#     답변: 도서관은 평일 오전 9시부터 오후 6시까지 운영됩니다. 주말에는 오전 10시부터 오후 5시까지 개방합니다. 공휴일에는 휴관입니다.
    
#     질문: {query}
#     답변:
#     """
#     prompt = f"Context: {context}\n\n{few_shot_example}"
    
    return generate_response(prompt)

In [29]:
# 사용 예시
query = "적응형학습이 뭐야?"
response = rag_query(query)
print(response)

적응형 학습이란 학생의 개별적인 요구, 선호도 및 학습 속도에 기반하여 교육 자료를 조정하는 학습 접근 방식을 말합니다. 이는 학생이 자신의 학습 속도대로 진행할 수 있도록 개인화된 학습 경험을 제공하며, 강점과 약점에 따라 콘텐츠를 맞춤화함으로써 학생들이 더 효과적으로 배울 수 있도록 돕습니다. 적응형 학습은 학생들이 특정 개념이나 기술을 마스터할 때까지 추가적인 지원과 자원을 제공할 수 있게 함으로써 개별 학생의 필요에 맞추어 교육 자료를 조정합니다. 주어진 컨텍스트 정보에는 적응형 학습에 대한 구체적인 언급이 없습니다.


In [25]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl

Looking in indexes: https://download.pytorch.org/whl


In [26]:
import torch
print(torch.cuda.is_available()) 

False


**임베딩한 데이터가 제대로 반영되지 않은 이유:**

1. 임베딩 품질: get_embedding 함수가 제대로 작동하지 않아 모든 문서에 대해 유사한 임베딩을 생성했을 수 있음
2. 인덱스 문제: Faiss 인덱스가 제대로 생성되지 않았거나, 검색이 제대로 작동하지 않을 수 있음
3. 컨텍스트 선택: k=3으로 설정되어 있어, 가장 유사한 3개의 문서만 선택됩니다. 이 값이 너무 작아 관련 문서가 선택되지 않았을 수 있음
4. 모델 응답: OLLAMA 모델이 주어진 컨텍스트를 무시하고 자체 지식에 기반하여 응답했을 수 있음
5. 데이터 처리 오류: 파일 읽기나 전처리 과정에서 오류가 발생해 일부 문서가 제대로 처리되지 않았을 수 있음

**해결해야 할 문제 8/19 마무리**
1. 답안이 항상 일정하게 나오지 않는다 ✔️
2. 그림 파일 임베딩이 제대로 안 됨 ✔️
3. 임베딩 안 되는 파일들 이유 찾아보기 ✔️
4. 효과적으로 임베딩하는 방법 찾아보기
5. 임베딩한 내용이 효과적으로 답안에 반영되도록 하는 방법 알아보기 🔺
   (지금은 자체적으로 답변해주는 경우도 많은 것 같음, 정확하지 않은 정보는 조금이라도 답변을 해주려는게 오히려 위험한 것 같기도 하다)
6. 형식 조정할 수 있는지 template 구조 살펴보기

**해결해야 할 문제 8/20 마무리**
1. 홈페이지 내용을 웹크롤링
2. 학사과정 페이지에 경영대학 파일 임베딩 (pdf), **표**나(페이지는 잘 가져와지는지) 그림은 어떻게 임베딩 하는지 방법 찾아보고, 결과 확인
   EEVE랑 결과 값이 연동?이 되는지 확인해보기, 구조 같은지
3. 웹페이지 한번에 가져오는 웹크롤링 라이브러리 확인